In [1]:
import pyttsx3
import datetime
from ipynb.fs.full.Time import *
#from ipynb.fs.full.Wiki import *
from ipynb.fs.full.Scrape import *
import speech_recognition as sr
import spacy
from datetime import datetime
import pytz
from geograpy import places
import socket

In [2]:
def speak(audio):
    engine.say(audio) 
    engine.runAndWait()

In [3]:
def is_connected():
    try:
        # connect to the host -- tells us if the host is actually
        # reachable
        socket.create_connection(("1.1.1.1", 53))
        return True
    except OSError:
        pass
    return False

In [4]:
def takeCommand_on():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        #r.dynamic_energy_threshold = True
        #r.energy_threshold = 400
        
        #r.pause_threshold = 2
        #r.dynamic_energy_adjustment_damping = 0.15
        #r.adjust_for_ambient_noise(source, duration=3)
        #r.dynamic_energy_adjustment_ratio = 1.5
        #rec=r.record(source, duration = None, offset = None)
        #b r.listen_in_background(source, callback) 
    
        print("Please wait. Calibrating google microphone...")  
        # listen for 5 seconds and create the ambient noise energy level  
        r.adjust_for_ambient_noise(source, duration=3)  
        print("Say something!")
        
        audio = r.listen(source)
        try:  
            rt=r.recognize_google(audio,language='en-in')
            return rt
        except sr.UnknownValueError:  
            speak("Sorry I did not get that")  
        except sr.RequestError as e:  
            speak("Sorry I did not get that")


In [5]:
def takeCommand_off():    
    r = sr.Recognizer()  
    with sr.Microphone() as source:  
        print("Please wait. Calibrating sphinx microphone...")  
        # listen for 5 seconds and create the ambient noise energy level  
        r.adjust_for_ambient_noise(source, duration=5)  
        print("Say something!")  
        audio = r.listen(source)
   
        # recognize speech using Sphinx  
        try:  
            rt=r.recognize_sphinx(audio)
            return rt
        except sr.UnknownValueError:  
            speak("Sorry I did not get that")  
        except sr.RequestError as e:  
            speak("Sorry I did not get that") 

In [6]:
def wishMe():
    hour = int(datetime.now().hour)
    if hour>=0 and hour<12:
        speak("Good Morning boss!")
    elif hour>=12 and hour<18:
        speak("Good Afternoon boss!")   
    else:
        speak("Good Evening boss!")  

In [7]:
engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[0].id)
nlp = spacy.load("en_core_web_lg")
places=[]
nouns=[]
lst=[]
propn=[]
lemmas=[]

i=False

In [8]:
def weather_check(statement):
  weather = nlp("Current weather in a city")
  statement = nlp(statement)
  min_similarity = 0.75

In [19]:
if __name__=="__main__":
    #wishMe()
    if is_connected():
        #rt=takeCommand_on()
        rt=input("Enter\n")
        online=True
    else:
        rt=takeCommand_off()
        rt=input("Enter\n")
        online=False
        
    #rt=input("Enter\n")#temporary
    doc = nlp(rt)
    for ent in doc.ents:
        if ent.label_=='GPE':
            places.append(ent.text)
    for token in doc:
        # Print the token and its part-of-speech tag
        print(token.text,"--->", token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_, token.is_alpha, token.is_stop)
        t_list=[token.text, token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_, token.is_alpha, token.is_stop]
        lemmas.append(token.lemma_)
        lst.append(t_list)        
        if token.pos_=='PROPN':
            propn.append(token)
        if token.tag_=='WRB' or token.tag_=='WP':
            i=True
#-----------------------------------------------ONLINE SPECIFIC-----------------------------------------------------------------       
    if i and online==True:
        oinfo=get_when(rt)
        if oinfo==1:
            results = wikipedia.summary(rt, sentences =2)
            #speak("According to Wikipedia")
            print(results)
            speak(results)
        
        elif type(oinfo)==list:
            output= oinfo[0] + oinfo[1] + " in the year "+ oinfo[2]
            speak(output)
            print(output)
        else:
            print(oinfo)
            speak(oinfo)
        print(oinfo)
        del oinfo 
        i=False 
#-------------------------------------------------COMMON------------------------------------------------------------------------




#--------------------------------------------------OFFLINE SPECIFIC----------------------------------------------------------------------
    if i and online==False:
        if 'time' in rt:
            if places:
                time=specific(places[0])
            else:
                time=local()
            speak(time)
            print(time)
            i=False

    lst.clear() 
    lemmas.clear()

Enter
who is the president of india
who ---> who PRON WP nsubj xxx True True
is ---> be AUX VBZ ROOT xx True True
the ---> the DET DT det xxx True True
president ---> president NOUN NN attr xxxx True False
of ---> of ADP IN prep xx True True
india ---> india NOUN NN pobj xxxx True False




NameError: name 'oinfo' is not defined